In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import requests

url = "https://raw.githubusercontent.com/daniel88516/diabetes-data/main/10k.csv"
s = requests.get(url).content
df_data = pd.read_csv(io.StringIO(s.decode('utf-8')))
df_data.drop(df_data[df_data['diabetes'] == 'No Info'].index, inplace=True) #沒有結果(diabetes)的資料沒用，直接刪掉
df_data.dropna(thresh=7, inplace=True) #篩掉有三個以上attribute缺失的資料

gender_map = {'Male':0 , 'Female':1  , 'Other':2}  #經過篩查之後才發現有Other
smoking_history_map = {'never':0 , 'former':1 , 'not current':2 , 'current':3  ,'ever':4 ,'No Info':0 }

#補齊資料
df_data['gender'] = df_data['gender'].map(gender_map)
df_data['age'].replace('No Info', df_data['age'].mean(), inplace=True)
df_data['smoking_history'] = df_data['smoking_history'].map(smoking_history_map)
df_data['hypertension'].replace('No Info', 0 , inplace=True)
df_data['heart_disease'].replace('No Info', 0 , inplace=True)
df_data['bmi'].replace('No Info', df_data['bmi'].mean() , inplace=True)
df_data['HbA1c_level'].replace('No Info', df_data['HbA1c_level'].mean() , inplace=True)
df_data['blood_glucose_level'].replace('No Info', df_data['blood_glucose_level'].mean() , inplace=True)

df_data

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,1,80.00,0,1,0,25.19,6.6,140.0,0.0
1,1,54.00,0,0,0,27.32,6.6,80.0,0.0
2,0,28.00,0,0,0,27.32,5.7,158.0,0.0
3,1,36.00,0,0,3,23.45,5.0,155.0,0.0
4,0,76.00,1,1,3,20.14,4.8,155.0,0.0
...,...,...,...,...,...,...,...,...,...
185894,0,61.00,0,0,0,29.66,9.0,160.0,1.0
185895,0,0.32,0,0,0,15.26,5.7,155.0,0.0
185896,0,50.00,0,0,0,27.32,6.1,130.0,0.0
185897,1,80.00,0,0,0,26.87,4.8,145.0,0.0


In [3]:
rows_with_nan = df_data[df_data.isnull().any(axis=1)].index
# 如果存在含有NaN值的行，印出位置
if len(rows_with_nan) > 0:
    print("資料中存在 NaN 值的行，行位置（行號）為:", rows_with_nan)
else:
    print("資料中不存在 NaN 值。")

資料中不存在 NaN 值。


In [4]:
from sklearn.model_selection import train_test_split
X = df_data.drop(labels=['diabetes'], axis=1)
X


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level
0,1,80.00,0,1,0,25.19,6.6,140.0
1,1,54.00,0,0,0,27.32,6.6,80.0
2,0,28.00,0,0,0,27.32,5.7,158.0
3,1,36.00,0,0,3,23.45,5.0,155.0
4,0,76.00,1,1,3,20.14,4.8,155.0
...,...,...,...,...,...,...,...,...
185894,0,61.00,0,0,0,29.66,9.0,160.0
185895,0,0.32,0,0,0,15.26,5.7,155.0
185896,0,50.00,0,0,0,27.32,6.1,130.0
185897,1,80.00,0,0,0,26.87,4.8,145.0


In [5]:
Y = df_data[['diabetes']]
Y

,diabetes
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
185894,1.0
185895,0.0
185896,0.0
185897,0.0


In [6]:
X_train , X_test ,Y_train , Y_test = train_test_split(X , Y , test_size=0.2 , random_state=42)
print('X_Training data shape:' , X_train.shape)
print('X_Test data shape:' , X_test.shape)
print('Y_Training data shape:' , Y_train.shape)
print('Y_Test data shape:' , Y_test.shape)
#用以確認訓練集和測試集有被分割正確

X_Training data shape: (148719, 8)
X_Test data shape: (37180, 8)
Y_Training data shape: (148719, 1)
Y_Test data shape: (37180, 1)


In [8]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import joblib

Model = LogisticRegression(random_state=0, max_iter=10000)
Model = Model.fit(X_train, Y_train)  # 用訓練集訓練模型
print('訓練集準確率:', Model.score(X_train, Y_train))
print('測試集準確率:', Model.score(X_test, Y_test))

file_path = './diabete_prediction_model.pkl'
joblib.dump(Model, file_path)



d:\Users\lenovo\anaconda3\envs\mnist-env\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


訓練集準確率: 0.9601530402974737
測試集準確率: 0.9604088219472835


['./diabete_prediction_model.pkl']

In [ ]:
import pandas as pd
import joblib

#!pip install anvil-uplink


# anvil UI
"""
import anvil.server
import anvil.mpl_util
anvil.server.connect("server_YZPTHBKSCHFB6WRMUGTLRCZ4-L6E5OGZPN6KFAUWX")

@anvil.server.callable
def predict_diabete(gender, age, hypertension, heart_disease ,smoking_history ,bmi ,HbA1c_level ,blood_glucose_level):
  prediction = Model.predict([[gender, age, hypertension, heart_disease ,smoking_history ,bmi ,HbA1c_level ,blood_glucose_level]])
  return prediction[0]
"""
model = joblib.load('/content/drive/My Drive/diabete_prediction_model.pkl')

# 請求用戶輸入八個特徵值
gender = input("請輸入性別 (Male:0 /Female:1 /Other:2 ): ")
age = float(input("請輸入年齡: "))
hypertension = float(input ("請輸入高血壓狀況  (無:0/有:1): "))
heart_disease = float(input("請輸入心臟病狀況  (無:0/有:1): "))
smoking_history = float(input("請輸入吸煙習慣 ( never:0 /former:1 /not current:2 /current:3 /ever:4 ): "))
bmi = float(input("請輸入BMI: "))
HbA1c_level = float(input("請輸入HbA1c水平: "))
blood_glucose_level = float(input("請輸入血糖水平: "))

# 將用戶輸入的特徵值組成 DataFrame
user_input = pd.DataFrame({
    'gender': [gender],
    'age': [age],
    'hypertension': [hypertension],
    'heart_disease': [heart_disease],
    'smoking_history': [smoking_history],
    'bmi': [bmi],
    'HbA1c_level': [HbA1c_level],
    'blood_glucose_level': [blood_glucose_level]
}, columns=['gender', 'age', 'hypertension', 'heart_disease', 'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level'])

prediction = model.predict(user_input)
result = "沒有糖尿病" if prediction[0] == 0 else "有糖尿病"
print("糖尿病預測：", result)






KeyboardInterrupt: Interrupted by user